In [1]:
from fcgb.cfg.precompiled import get_llm, get_checkpointer
from fcgb.tools.spectools import PhantomResearcherSpecTool, JobHandlerSpecTool, JobHandler
from fcgb.cfg.chat_inputs_spec import JobHanlderConfig

## Fake LLM

In [2]:
fake_llm = get_llm(llm_model='fake')
memory = get_checkpointer(checkpointer_mode='local', mode='test')

In [3]:
tool_containers = [
    PhantomResearcherSpecTool(llm=fake_llm, memory=memory)
]

In [4]:
job_handler = JobHandlerSpecTool(
    llm=fake_llm,
    tool_containers=tool_containers,
    memory=memory,
)

In [5]:
outputs = job_handler.run(job_description='some job description')
print(outputs)

{'thread_id': '3e132091992a4d38af5ab16923d05857', 'output': 'Fake LLM response 2'}


In [6]:
job_state = job_handler.get_state(thread_id=outputs['thread_id'])

In [7]:
job_state

{'job_description': 'some job description',
 'turns': 2,
 'messages': [SystemMessage(content='Your purpose is to conduct a single job that will be provided to you by the user.\nYou are also equipped with a set of tools that you can use to complete the job.\n\nPut attention on the job description as well as output format if specified.\nYou have maximum of 4 turns to call the tools and complete the job.\nYou can call the same tool multiple times (even within single turn) with different parameters to solve the job.\nIf you will exceed the maximum number of turns, tools will be disabled and you will have to answer with the information you have at that moment.', additional_kwargs={}, response_metadata={}, id='350022a1-bcaa-4180-a285-ebbcc6d2540d'),
  HumanMessage(content='Process the following job:\n\nsome job description\n\nBe aware of provided restrictions and output format.\nStick to the job description and do not provide any additional information or context.\nIf there is any additional

In [8]:
for msg in job_state['messages']:
    msg.pretty_print()

================================ System Message ================================

Your purpose is to conduct a single job that will be provided to you by the user.
You are also equipped with a set of tools that you can use to complete the job.

Put attention on the job description as well as output format if specified.
You have maximum of 4 turns to call the tools and complete the job.
You can call the same tool multiple times (even within single turn) with different parameters to solve the job.
If you will exceed the maximum number of turns, tools will be disabled and you will have to answer with the information you have at that moment.
================================ Human Message =================================

Process the following job:

some job description

Be aware of provided restrictions and output format.
Stick to the job description and do not provide any additional information or context.
If there is any additional data provided use it for your answer. You can also pass

In [9]:
for tool_thread in job_state['tools_threads']:
    tool_state = job_handler.get_state(thread_id=tool_thread)
    print(tool_state)
    print('-'*50)

{'job': 'Fake string xcror', 'restrictions': 'Fake string fdppx', 'output_format': 'Fake string axtqd', 'data': 'Fake string rrsgl', 'messages': [SystemMessage(content='Your purpose is to mimic a research tool that uses external resources like web pages, research papers, wikipedia etc. to answer questions and provide information.\nIf you will be asked to use those external resources use your best knowledge to provide the most accurate and relevant information whitout using any external resources but within your answer you can pretend that you are using them.\n\nYou are a part of a testing tool so accuracy is not that important but try to be as accurate as possible.\n\nAvoid any unnecessary comments that are not a integral part of the answer.\n\n', additional_kwargs={}, response_metadata={}, id='bbc613d3-6cfc-47a4-b3bb-12a7c611c064'), HumanMessage(content="Provide an answer for a research job which is:\n**research job**: Fake string xcror\n\nWhen writing your answer be aware of provided

In [10]:
tool_thread = job_state['tools_threads'][0]
tool_state = job_handler.get_state(thread_id=tool_thread)
for msg in tool_state['messages']:
    msg.pretty_print()

================================ System Message ================================

Your purpose is to mimic a research tool that uses external resources like web pages, research papers, wikipedia etc. to answer questions and provide information.
If you will be asked to use those external resources use your best knowledge to provide the most accurate and relevant information whitout using any external resources but within your answer you can pretend that you are using them.

You are a part of a testing tool so accuracy is not that important but try to be as accurate as possible.

Avoid any unnecessary comments that are not a integral part of the answer.


================================ Human Message =================================

Provide an answer for a research job which is:
**research job**: Fake string xcror

When writing your answer be aware of provided restrictions:
**restrictions**: Fake string fdppx

There is also specified output format of your answer:
**output format**: Fa

In [27]:
fake_tooled_llm = fake_llm.bind_tools([container.get_tool() for container in tool_containers], parallel_tool_calls=False)

In [35]:
fake_tooled_llm().tool_calls

[]

## Real LLM

In [41]:
inputs = {
    'job_description': """
    job: Explore research papers to identify and list the most commonly cited and foundational Chain-of-Thought prompting variants
    motivation: To establish a baseline understanding of the well-known and widely adopted CoT variants, providing a strong foundation for the research task
    restrictions: Focus on variants that are widely recognized and have substantial documentation or research backing them. Avoid niche or experimental variants that lack sufficient information. Also do not include fine-tuning methods or techniques that are not directly related to prompting.
    output_format: A list of variant names, each with a brief (1-2 sentence) description of its core idea.
    """
}

In [28]:
llm = get_llm(llm_model='google')
memory = get_checkpointer(checkpointer_mode='local', mode='test')

In [29]:
tool_containers = [
    PhantomResearcherSpecTool(llm=llm, memory=memory)
]

In [59]:
job_handler = JobHandler(
    llm=llm,
    tool_containers=tool_containers,
    initial_messages_spec=JobHanlderConfig.initial_messages_spec,
    internal_messages_spec=JobHanlderConfig.internal_messages_spec,
    memory=memory,
    init_values=JobHanlderConfig.init_values,
    prompt_manager_spec=JobHanlderConfig.prompt_manager_spec,
    global_inputs=JobHanlderConfig.global_inputs
)

In [60]:
outputs = job_handler.run(**inputs)
print(outputs)

{'thread_id': 'b7b5641305764ecfa36864f1f71366a4', 'output': 'Foundational Chain-of-Thought Prompting Variants:\n\n*   **Standard Chain-of-Thought Prompting:** This involves providing a few example questions along with their step-by-step reasoning processes in the prompt, guiding the language model to generate similar reasoning steps for new, unseen questions. The core idea is to elicit intermediate reasoning steps, leading to improved accuracy in complex reasoning tasks.\n\n*   **Zero-Shot Chain-of-Thought Prompting:** This variant leverages the "Let\'s think step by step" prompt, or similar variations, to encourage the model to generate a chain of thought without requiring any explicit examples in the prompt. It relies on the model\'s pre-existing knowledge and reasoning abilities.\n\n*   **Self-Consistency for Chain-of-Thought Prompting:** This approach involves generating multiple independent reasoning paths for a single question using CoT prompting. The final answer is then determi

In [61]:
job_state = job_handler.get_state(thread_id=outputs['thread_id'])

In [62]:
job_state

{'job_description': '\n    job: Explore research papers to identify and list the most commonly cited and foundational Chain-of-Thought prompting variants\n    motivation: To establish a baseline understanding of the well-known and widely adopted CoT variants, providing a strong foundation for the research task\n    restrictions: Focus on variants that are widely recognized and have substantial documentation or research backing them. Avoid niche or experimental variants that lack sufficient information. Also do not include fine-tuning methods or techniques that are not directly related to prompting.\n    output_format: A list of variant names, each with a brief (1-2 sentence) description of its core idea.\n    ',
 'turns': 2,
 'messages': [SystemMessage(content='Your purpose is to conduct a single job that will be provided to you by the user.\nYou are also equipped with a set of tools that you can use to complete the job.\n\nPut attention on the job description as well as output format 

In [63]:
for msg in job_state['messages']:
    msg.pretty_print()

================================ System Message ================================

Your purpose is to conduct a single job that will be provided to you by the user.
You are also equipped with a set of tools that you can use to complete the job.

Put attention on the job description as well as output format if specified.
You have maximum of 4 turns to call the tools and complete the job.
You can call the same tool multiple times (even within single turn) with different parameters to solve the job.
If you will exceed the maximum number of turns, tools will be disabled and you will have to answer with the information you have at that moment.
================================ Human Message =================================

Process the following job:


    job: Explore research papers to identify and list the most commonly cited and foundational Chain-of-Thought prompting variants
    motivation: To establish a baseline understanding of the well-known and widely adopted CoT variants, providi

In [64]:
for tool_thread in job_state['tools_threads']:
    tool_state = job_handler.get_state(thread_id=tool_thread)
    print(tool_state)
    print('-'*50)

{'job': 'Explore research papers to identify and list the most commonly cited and foundational Chain-of-Thought prompting variants', 'motivation': 'To establish a baseline understanding of the well-known and widely adopted CoT variants, providing a strong foundation for the research task', 'restrictions': 'Focus on variants that are widely recognized and have substantial documentation or research backing them. Avoid niche or experimental variants that lack sufficient information. Also do not include fine-tuning methods or techniques that are not directly related to prompting.', 'output_format': 'A list of variant names, each with a brief (1-2 sentence) description of its core idea.', 'messages': [SystemMessage(content='Your purpose is to mimic a research tool that uses external resources like web pages, research papers, wikipedia etc. to answer questions and provide information.\nIf you will be asked to use those external resources use your best knowledge to provide the most accurate a